In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], "src"))

## create scene builder

In [2]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")

## init combined robot config

In [3]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7,  ((0,0,0), (0,0,0)),
                INDY_IP)]
              , connection_list=[False])
gscene = s_builder.create_gscene(crob)

connection_list
[False]


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## init planning scene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

### planning pipeline & UI

In [5]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


## add environment

In [6]:
from pkg.geometry.geometry import *

   Use a production WSGI server instead.
 * Debug mode: off


In [7]:
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.6,0.6,0.01), (0.4,0,-0.005), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=False)
floor = gscene.create_safe(GEOTYPE.BOX, "floor_viscol", "base_link", (3,3,0.01), (0,0,-0.006), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.4,0.3,-0.005), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=True, collision=False)

Please create a subscriber to the marker


In [8]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add box geometries

In [9]:
gbox1 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box1", link_name="base_link", 
                                dims=(0.05,0.05,0.05), center=(0.4,-0.3,0.025), rpy=(0,0,0), 
                                color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)
gbox2 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box2", link_name="base_link", 
                                dims=(0.05,0.05,0.05), center=(0.41,0.23,0.025), rpy=(0,0,0), 
                                color=(0.3,0.7,0.3,1), display=True, collision=True, fixed=False)
gbox3 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box3", link_name="base_link", 
                                dims=(0.05,0.05,0.05), center=(0.33,0.31,0.025), rpy=(0,0,0), 
                                color=(0.3,0.3,0.7,1), display=True, collision=True, fixed=False)
gbox4 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box4", link_name="base_link", 
                                dims=(0.05,0.05,0.05), center=(0.4,0.0,0.025), rpy=(0,0,0), 
                                color=(0.3,0.7,0.3,1), display=True, collision=True, fixed=False)
gbox5 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box5", link_name="base_link", 
                                dims=(0.05,0.05,0.05), center=(0.4,0.3,0.025), rpy=(0,0,0), 
                                color=(0.3,0.3,0.7,1), display=True, collision=True, fixed=False)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import BoxObject, CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask

In [14]:
box1 = pscene.create_object("box1", "box1", BoxObject, hexahedral=True)
box2 = pscene.create_object("box2", "box2", BoxObject, hexahedral=True)
box3 = pscene.create_object("box3", "box3", BoxObject, hexahedral=True)
box4 = pscene.create_object("box4", "box4", BoxObject, hexahedral=True)
box5 = pscene.create_object("box5", "box5", BoxObject, hexahedral=True)

## Grasp checker

In [15]:
from pkg.planning.filtering.grasp_filter import GraspChecker
gcheck = GraspChecker(pscene, end_link_couple_dict= {"indy0_tcp": ["indy0_tcp", "indy0_link6"], "base_link":["base_link"]})

## Reach SVM

In [16]:
from pkg.planning.filtering.reach_filter import ReachChecker
rcheck = ReachChecker(pscene)

## motion planner

In [ ]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, [gcheck, rcheck])
mplan.update_gscene()
ppline.set_motion(mplan)

## set initial state

In [ ]:
initial_state = pscene.update_state(crob.home_pose)

# ===== TESTING TASK PLANNERS FROM HERE =====

## Test RRT

In [ ]:
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_sampler(tplan)

In [ ]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
mplan.reset_log(True)
for _ in range(5):
    with GlobalTimer.instance().block("full"):
        goal_nodes = [("goal", "floor", "floor", "floor", "floor")]
        ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=True, timeout=1)
        schedules = ppline.tplan.find_schedules()
        schedules_sorted = ppline.sort_schedule(schedules)
        snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])
print(GlobalTimer.instance())

In [21]:
for k,v in mplan.result_log.items():
    print("{}: {}% ({})".format(k, np.mean(v)*100, len(v)))

GraspChecker: nan% (0)
planning: nan% (0)
ReachChecker: nan% (0)


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:85: RuntimeWarning:

invalid value encountered in double_scalars



In [22]:
print(GlobalTimer.instance())

full: 	40701.0 ms/5 = 8140.163 ms (2094.921/15127.13)



In [23]:
time.sleep(1)
snode_schedule = ppline.add_return_motion(snode_schedule)
ppline.play_schedule(snode_schedule, period=0.01)

## test BiRRT

In [ ]:
from pkg.planning.task.rrt_bi import TaskBiRRT
tplan = TaskBiRRT(pscene, gcheck)
tplan.prepare()
ppline.set_sampler(tplan)

In [ ]:
tplan = TaskBiRRT(pscene, gcheck)
tplan.prepare()
ppline.set_sampler(tplan)

In [ ]:
mplan.reset_log(True)
for _ in range(5):
    with GlobalTimer.instance().block("full"):
        goal_nodes = [("goal", "floor", "floor", "floor", "floor")]
        ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=True, timeout=1)
        schedules = ppline.tplan.find_schedules()
        schedules_sorted = ppline.sort_schedule(schedules)
        snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])
        
print(GlobalTimer.instance())

In [22]:
for k,v in mplan.result_log.items():
    print("{}: {}% ({})".format(k, np.mean(v)*100, len(v)))

GraspChecker: nan% (0)
planning: nan% (0)
ReachChecker: nan% (0)


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:85: RuntimeWarning:

invalid value encountered in double_scalars



In [23]:
print(GlobalTimer.instance())

full: 	58050.0 ms/5 = 11610.068 ms (4045.398/22176.655)



## test BiRRT with SWAP

In [ ]:
from pkg.planning.task.rrt_bi import TaskBiRRT
tplan = TaskBiRRT(pscene, gcheck, flag_swap=True)
tplan.prepare()
ppline.set_sampler(tplan)

In [ ]:
mplan.reset_log(True)
for _ in range(5):
    with GlobalTimer.instance().block("full"):
        goal_nodes = [("goal", "floor", "floor", "floor", "floor")]
        ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=True, timeout=1)
        schedules = ppline.tplan.find_schedules()
        schedules_sorted = ppline.sort_schedule(schedules)
        snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])
        
print(GlobalTimer.instance())

In [24]:
print(GlobalTimer.instance())

full: 	84294.0 ms/5 = 16858.723 ms (7663.828/30542.501)



In [58]:
snode_schedule = ppline.add_return_motion(snode_schedule)
ppline.play_schedule(snode_schedule, period=0.05)

In [97]:
ppline.play_schedule(snode_schedule, period=0.05)